In [1]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, T5ForConditionalGeneration, T5Tokenizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
import nltk

In [2]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained('try_new/Pegasus/Tokenizer/')
model = PegasusForConditionalGeneration.from_pretrained('try_new/Pegasus/Model/').to(torch_device)


# tokenizer = T5Tokenizer.from_pretrained('try_new/tokenizer/')
# model = T5ForConditionalGeneration.from_pretrained('try_new/model/').to(torch_device)

f:\paraphrase\paraphraseenv\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def get_response(sentence):
    input_text = sent_tokenize(sentence)
    if len(input_text[-1]) < 5:
        input_text = input_text[:-1]
    
    encoding = tokenizer.batch_encode_plus(input_text, padding=True, pad_to_multiple_of=16, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(torch_device), encoding["attention_mask"].to(torch_device)
    translated = model.generate(input_ids=input_ids, 
                                attention_mask=attention_masks,
                                do_sample=True,
                                min_length=max([len(i.split()) for i in input_text]) - 2,
                                max_length=max([len(i.split()) for i in input_text]) + 5,
                                top_k=120,
                                top_p=0.95,
                                temperature=0.98,
                                early_stopping=True,
                                num_return_sequences=1,
                                no_repeat_ngram_size = 3
                               )
    
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    unmod = ' '.join(tgt_text)
    final = ' '.join(tgt_text)
    
    tokens = nltk.word_tokenize(final)
    tag = nltk.pos_tag(tokens)
    tags_only = [i[1] for i in tag]
    selected = [i for i, x in enumerate(tags_only) if x == "NN" or x=="JJ"]
    syn = []

    for i in selected:
        try:
            syn.append(wordnet.synsets(tokens[i])[-1].lemmas()[-1].name())
        except:
            syn.append(tokens[i])
            pass
     
    j = 0
    for i in selected:
        tokens[i] = syn[j]
        j+=1
    
    final = ' '.join(tokens)
    
    return final , unmod

In [4]:
sentence = "The Android 11 update comes with version number 3.150 and requires a download of 1.5GB. It bumps up the Android security patch level on the Nokia 4.2 to March 2021 and brings new features to the smartphone, such as chat bubbles and one-time app permissions."

target_1, target_2 = get_response(sentence)

print(sentence)
print()
print('Main')
print(target_1)
print()
print('Unmod')
print(target_2)

The Android 11 update comes with version number 3.150 and requires a download of 1.5GB. It bumps up the Android security patch level on the Nokia 4.2 to March 2021 and brings new features to the smartphone, such as chat bubbles and one-time app permissions.

Main
The version keep_down of the update is 3.150 and requires a download of 1.5 gigaflops of disk space , according to the site . It brings new features to the ring , such as chat bubbles and sometime app permit , as well as raising the security_measures patch level on the ring .

Unmod
The version number of the update is 3.150 and requires a download of 1.5 gigaflops of disk space, according to the website. It brings new features to the phone, such as chat bubbles and one-time app permission, as well as raising the security patch level on the phone.


In [17]:
max([len(i.split()) for i in sentence.split('.')])

27

In [6]:
sentence = "The Android 11 update comes with version number 3.150 and requires a download of 1.5GB. It bumps up the Android security patch level on the Nokia 4.2 to March 2021 and brings new features to the smartphone, such as chat bubbles and one-time app permissions."
input_text = sent_tokenize(sentence)
if len(input_text[-1]) < 5:
    input_text = input_text[:-1]
tokenizer.batch_encode_plus(input_text,padding=True, pad_to_multiple_of=16, return_tensors="pt")

ModuleAttributeError: 'T5ForConditionalGeneration' object has no attribute 'batch_encode_plus'

In [49]:
b = model.generate(input_ids=a['input_ids'], 
                attention_mask=a['attention_mask'],
                do_sample=True,
                min_length=max([len(i.split()) for i in sentence.split('.')]) - 2,
                max_length=max([len(i.split()) for i in sentence.split('.')]) + 5,
                top_k=120,
                top_p=0.95,
                temperature=0.98,
                early_stopping=True,
                num_return_sequences=1,
                no_repeat_ngram_size = 3
               )
c = tokenizer.batch_decode(b, skip_special_tokens=True)
' '.join(c)

"It is easy to get starstruck by the Pro sibling, but the OnePlus 9 is a capable alternative at a lower cost. The base version of the 9 Pro is not available in the US, so you're looking at a huge gap. You can have the vanilla phone for $730 or the 12/128 gigabyte pro for $1,070, whichever is less."

In [50]:
sentence

'While it’s easy to get starstruck by its Pro sibling, the OnePlus 9 is a capable alternative at a lower cost. You’re looking at quite a gap, especially in the US where (for some unknown reason), the base 8/128 GB version of the 9 Pro is not available. This means you can have the vanilla phone for $730 or the 12/256 GB Pro for $1,070'

In [13]:
sentence

'While it’s easy to get starstruck by its Pro sibling, the OnePlus 9 is a capable alternative at a lower cost. You’re looking at quite a gap, especially in the US where (for some unknown reason), the base 8/128 GB version of the 9 Pro is not available. This means you can have the vanilla phone for $730 or the 12/256 GB Pro for $1,070'

In [ ]:
import random

In [14]:
tok1 = nltk.word_tokenize(target)
tag2 = nltk.pos_tag(tok1)

In [28]:
temp = [i[1] for i in tag2]

In [52]:
temp_2 = [i for i, x in enumerate(temp) if x == "NN" or x=="JJ"]

In [53]:
temp_2

[2, 4, 16, 17, 20, 21, 28, 39, 40, 46]

In [54]:
[dictionary.synonym(tok1[i])[-1] for i in temp_2]

['quint',
 'painless',
 'cut-rate',
 'default',
 'ground',
 'approximation',
 'gettable',
 'immense',
 'spread',
 'speakerphone']

In [55]:
[tok1[i] for i in temp_2]

['sibling',
 'easy',
 'cheaper',
 'alternative',
 'base',
 'version',
 'available',
 'huge',
 'gap',
 'phone']

In [45]:
try:
    dictionary.synonym('aaaa')

aaaa has no Synonyms in the API


In [36]:
max([len(i.split()) for i in context])

min([len(i.split()) for i in context])

17

In [19]:
# model.save_pretrained('Pegasus/Model/')

In [56]:
tokens = nltk.word_tokenize(target)
tag = nltk.pos_tag(tokens)
tags_only = [i[1] for i in tag]
selected = [i for i, x in enumerate(tags_only) if x == "NN" or x=="JJ"]
syn = []
for i in selected:
    temp = dictionary.synonym(tokens[i])

    if type(temp) == list:
        syn.append(temp[-1])
    else:
        syn.append(tokens[i])

j = 0
for i in selected:
    tokens[i] = syn[j]
    j+=1

final = ' '.join(tokens)

In [57]:
final

'The Pro quint is painless to get starstruck by , but the OnePlus 9 is a cut-rate default . The ground approximation of the 9 Pro is not gettable in the US , so you are looking at a immense spread . You can get the speakerphone for $ 730 or the Pro for $ 1,070 , whichever is less .'

In [58]:
target

'The Pro sibling is easy to get starstruck by, but the OnePlus 9 is a cheaper alternative. The base version of the 9 Pro is not available in the US, so you are looking at a huge gap. You can get the phone for $730 or the Pro for $1,070, whichever is less.'

In [59]:
sentence

'While it’s easy to get starstruck by its Pro sibling, the OnePlus 9 is a capable alternative at a lower cost. You’re looking at quite a gap, especially in the US where (for some unknown reason), the base 8/128 GB version of the 9 Pro is not available. This means you can have the vanilla phone for $730 or the 12/256 GB Pro for $1,070'

In [1]:
import nltk

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     f:\paraphrase\paraphraseenv\share\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from nltk.corpus import wordnet

In [6]:
wordnet.synsets("cheapest")[-1].lemmas()[-1].name()

'chintzy'

In [7]:
a = 'While it’s easy to get starstruck by its Pro sibling, the OnePlus 9 is a capable alternative at a lower cost. You’re looking at quite a gap, especially in the US where (for some unknown reason), the base 8/128 GB version of the 9 Pro is not available. This means you can have the vanilla phone for $730 or the 12/256 GB Pro for $1,070'

In [10]:
tokens = nltk.word_tokenize(a)
tag = nltk.pos_tag(tokens)
tags_only = [i[1] for i in tag]
selected = [i for i, x in enumerate(tags_only) if x == "NN" or x=="JJ"]
syn = []
for i in selected:
    try:
        syn.append(wordnet.synsets(tokens[i])[-1].lemmas()[-1].name())
    except:
        syn.append(tokens[i])
        pass

j = 0
for i in selected:
    tokens[i] = syn[j]
    j+=1

b = ' '.join(tokens)

print(a)
print()
print(b.replace(' ’ ','\''))

While it’s easy to get starstruck by its Pro sibling, the OnePlus 9 is a capable alternative at a lower cost. You’re looking at quite a gap, especially in the US where (for some unknown reason), the base 8/128 GB version of the 9 Pro is not available. This means you can have the vanilla phone for $730 or the 12/256 GB Pro for $1,070

While it'S soft to get starstruck by its Pro sib , the OnePlus 9 is a capable alternative at a lower cost . You'ray looking at quite a breach , especially in the US where ( for some unknown reason ) , the base 8/128 GB version of the 9 Pro is not useable . This means you can have the vanilla ring for $ 730 or the 12/256 GB Pro for $ 1,070


In [56]:
from nltk.tokenize import sent_tokenize
text = "The Android 11 update comes with version number 3.150 and requires a download of 1.5GB. It bumps up the Android security patch level on the Nokia 4.2 to March 2021 and brings new features to the smartphone, such as chat bubbles and one-time app permissions."

In [58]:
print(sent_tokenize(text))

['The Android 11 update comes with version number 3.150 and requires a download of 1.5GB.', 'It bumps up the Android security patch level on the Nokia 4.2 to March 2021 and brings new features to the smartphone, such as chat bubbles and one-time app permissions.']
